Reference - article - https://www.analyticsvidhya.com/blog/2022/01/youtube-summariser-mini-nlp-project/

In [1]:
pip install youtube-transcript-api

Note: you may need to restart the kernel to use updated packages.


In [2]:
import youtube_transcript_api
from youtube_transcript_api import YouTubeTranscriptApi
import nltk
import re
from nltk.corpus import stopwords
import sklearn 
from sklearn.feature_extraction.text import TfidfVectorizer

ModuleNotFoundError: No module named 'sklearn'

In [6]:
! pip install pytube -q

In [7]:
from pytube import YouTube

In [ ]:
# !pip uninstall pyarrow
# !pip install pyarrow

In [ ]:
!pip install huggingsound -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 214.3/214.3 kB 29.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 776.3/776.3 MB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 21.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 59.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 66.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.2.1+cu121 requires torch==2.2.1, but you have torch 1.12.1 which is incompatible.
torchdata 0.7.1 requires torch>=2, but you have torch 1.12.1 which is incom

In [ ]:
from huggingsound import SpeechRecognitionModel

In [ ]:
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
device

'cuda'

In [ ]:
import librosa

In [ ]:
import soundfile as sf

In [ ]:
link = "https://www.youtube.com/watch?v=Y8Tko2YC5hA" #with subtitle
# link = "https://www.youtube.com/watch?v=8ZT_Q_B_qbM" age-restricted
# link = "https://youtu.be/RIJ2Jclv9Dg?si=SPwk1Dhw3W2EeMwe" # without subtitle
unique_id = link.split("=")[-1]

In [ ]:
from youtube_transcript_api import TranscriptsDisabled

try:
  sub = YouTubeTranscriptApi.get_transcript(unique_id)
  subtitle = " ".join([x['text'] for x in sub])
except TranscriptsDisabled as e:
  print("Subtitles are disabled for this video. You may need to handle this case differently.")
  # Your alternative code or error handling mechanism goes here

  # Download YouTube Video's Audio
  yt = YouTube(link)
  # Filter and get the first audio-only stream in mp4 format
  audio_stream = yt.streams.filter(only_audio=True, file_extension='mp4').first()

  # Download the audio stream
  audio_stream.download(filename='ytaudio.mp4')
  ! ffmpeg -i ytaudio.mp4 -acodec pcm_s16le -ar 16000 ytaudio.wav

  # English ASR with HuggingSound
  model = SpeechRecognitionModel("jonatasgrosman/wav2vec2-large-xlsr-53-english", device = device)

  # Audio Chunking
  input_file = '/content/ytaudio.wav'
  print(librosa.get_samplerate(input_file))

  # Stream over 30 seconds chunks rather than load the full file
  stream = librosa.stream(
      input_file,
      block_length=30,
      frame_length=16000,
      hop_length=16000
  )
  for i,speech in enumerate(stream):
    sf.write(f'{i}.wav', speech, 16000)

  # Audio Transcription / ASR / Speech to Text
  audio_path =[]
  for a in range(i+1):
    audio_path.append(f'/content/{a}.wav')
  transcriptions = model.transcribe(audio_path)
  subtitle = ' '
  for item in transcriptions:
    subtitle += ''.join(item['transcription'])
  len(subtitle)

# **Summarization using BART**

In [ ]:
!pip install transformers

In [ ]:
import transformers
from transformers import BartTokenizer, BartForConditionalGeneration

In [ ]:
!pip install nltk


In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("Mr-Vicky-01/Bart-Finetuned-conversational-summarization")
model = AutoModelForSeq2SeqLM.from_pretrained("Mr-Vicky-01/Bart-Finetuned-conversational-summarization")

def generate_summary(text):
    inputs = tokenizer([text], max_length=102400, return_tensors='pt', truncation=True)
    summary_ids = model.generate(inputs['input_ids'], max_new_tokens=10000, do_sample=False)
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary

text_to_summarize = subtitle
summary = generate_summary(text_to_summarize)

print(summary)

Python is the world's fastest growing and most popular programming language. People from different disciplines use Python for a variety of different tasks, such as data analysis and visualization, artificial intelligence and machine learning. Python is a multi-purpose language with a simple, clean, and beginner-friendly syntax. Python developers earn a whopping 116,000 dollars a year. Mosh has a couple of Python tutorials for beginners.


In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import nltk
nltk.download('punkt')

tokenizer = AutoTokenizer.from_pretrained("Mr-Vicky-01/Bart-Finetuned-conversational-summarization")
model = AutoModelForSeq2SeqLM.from_pretrained("Mr-Vicky-01/Bart-Finetuned-conversational-summarization")

def generate_summary(text):
    inputs = tokenizer([text], max_length=102400, return_tensors='pt', truncation=True)
    summary_ids = model.generate(inputs['input_ids'], max_new_tokens=10000, do_sample=False)
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary

text_to_summarize = subtitle
summary = generate_summary(text_to_summarize)

# Split the summary into individual sentences
sentences = nltk.sent_tokenize(summary)

# Print each sentence on a new line
for sentence in sentences:
    print(sentence)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Python is the world's fastest growing and most popular programming language.
People from different disciplines use Python for a variety of different tasks, such as data analysis and visualization, artificial intelligence and machine learning.
Python is a multi-purpose language with a simple, clean, and beginner-friendly syntax.
Python developers earn a whopping 116,000 dollars a year.
Mosh has a couple of Python tutorials for beginners.


In [ ]:
tokenizer = BartTokenizer.from_pretrained('facebook/bart-large-cnn')
model = BartForConditionalGeneration.from_pretrained('facebook/bart-large-cnn')

In [ ]:
input_tensor = tokenizer.encode( subtitle, return_tensors="pt", max_length=512)


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


In [ ]:
outputs_tensor = model.generate(input_tensor, max_length=160, min_length=120, length_penalty=2.0, num_beams=4, early_stopping=True)
outputs_tensor


tensor([[    2,     0, 17781,    10,  4069,     9,  5886, 31538,   895,  9611,
         18711,     8,  8276,    59,    80,   722,    71,   110,  5573,  5820,
             4,    83, 23611,    19,   143, 17145,  4297,  1334, 17589,  5806,
           196,    50,  7756,   873, 12158,   817,    13,    10,   372, 17687,
             4,   318,    47,   214, 10759,  1588,  1070,  1095,   409,    31,
         31130,     8,    62,    13,    41, 15162,  1386,     4,    83,   543,
            12,  3983,  6691,  8380, 35455,    19,   455,  5886,   313,   293,
             8,  7983,  9649,   118,   181,  3119,    16,    67,    10,   205,
         17687,     4,  1599,    75,  4309,     7, 17687,    77,    47,   214,
            15,     5,   300,  5113,    10, 17687,  3298,  4740,   455,     9,
         15092,     8,  3841,    12, 30929,    50,    10, 10894,    12,  4901,
          4066,    11,  8276,     4,  6893,    13,    65,    19,    23,   513,
           799, 14080,     9,  8276,     8,   292, 1

In [ ]:
# print(tokenizer.decode(outputs_tensor[0]))

In [ ]:
output_text = tokenizer.decode(outputs_tensor[0])

# Split the output text into multiple lines based on a specific character or length
# For example, splitting at every period ('.') or after a certain number of characters
lines = output_text.split('.')
# Or split by a certain length
# lines = [output_text[i:i+80] for i in range(0, len(output_text), 80)]

# Print each line separately
for line in lines:
    print(line.strip())  # strip to remove extra spaces or newlines


</s><s>Have a combination of fat carbahydrates and protein about two hours after your las meal
A banana with any nutbutter ulmoned or pinabutter makes for a great snack
If you're constipated stay away from bananas and up for an apple instead
A hard-boiled egg mashed with full fat manes and miniholi pita is also a good snack
Don't forget to snack when you're on the gotake a snack baggy full of nuts and dry-fruit or a nutrition-bar rich in protein
Look for one with at least eight grams of protein and five grams of fat
</s>
